In [1]:
import numpy as np
import pandas as pd
import os
import importlib as ipl
import scipy.io as io

## 0. Read .mat files


In [2]:
file_mat = '/Users/juanita/Documents/PhD/Data/sample_data/2018-08-07-width1p7-trimmed_forPython.mat'
file_meta = '/Users/juanita/Documents/PhD/Data/sample_data/storedMetaData.mat'

In [3]:
mat = io.loadmat(file_mat)
meta = io.loadmat(file_meta)

In [4]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'T', 'final_struct'])

In [5]:
dat = mat['final_struct']

dtypes of structures are "unsized objects". SciPy reads in structures as structured NumPy arrays of dtype object <br>
The size of the array is the size of the structure array, not the number
elements in any particular field. The shape defaults to 2-dimensional.

In [39]:
time = mat['T'].flatten()

In [7]:
mtype = dat.dtype

## 1. Create dictionary with the keys being the fields in the mat struct 

Fields contained in the matlab structure: <br>
1. X
2. Y
3. Area
4. Intensity
5. Frame
6. Eccentricity
7. Major Axis
8. Minor Axis
9. Ang
10. Track ID

In [8]:
ndata = {n: dat[n] for n in mtype.names}

In [9]:
columns = [n for n, v in ndata.items()]

In [10]:
columns = np.roll(columns,1)

## 2. Create dataframe 

### Structure of data frame:

Each row contains the time series measurements of a single particle. <br>
movieID 1-10 -> fluc <br>
movieID 11-20 -> stable low <br>
movieID 21-30 -> stable average <br>
movieID 31-40 -> stable high <br>

In [11]:
data = pd.DataFrame(np.concatenate([ndata[c] for c in columns], axis=1),
                  columns=columns)

The dataframe contains ndarrays that can be flatten for ease of use 

In [12]:
data = data.applymap(lambda x: x.flatten())

Get the movie ID out of the numpy array

In [13]:
data['movieID'] = data['movieID'].apply(lambda x: x.item(0))

The track ID is repeated the same number of times as there are items in the measurement numpy arrays. Only one number is needed

The TrackID is the identifier of each particle (can also be named track) before cleaning the data. The particleID is the number that I gave the identifier of each particle/track after cleaning the data

In [14]:
data['TrackID'] = data['TrackID'].apply(lambda x: np.unique(x)[0])

In [15]:
data['particleID'] = data['particleID'].apply(lambda x: np.unique(x)[0])

In the cells below, the numpy arrays will be unpacked in such a way that each row will be a single measurement. The index identifiers will be the movieID, the trackID, particleID in movie and a unique ID

First, set the identifiers as indeces

In [16]:
data2 = data.set_index(['particleID','movieID', 'TrackID'])

Second, loop through each column and convert the numpy arrays to a series. Then, use the function stack() to create a single-level column. The variable unnested_list contains a list of all the series. Use this series in combination with the keys from the original dataframe to create a new unstacked dataframe!

In [17]:
unnested_list = []
for col in data2.columns:
    unnested_list.append(data2[col].apply(pd.Series).stack())
data_stacked = pd.concat(unnested_list, axis=1, keys=data2.columns)

In [18]:
data_stacked.reset_index(inplace=True)

In [19]:
data_stacked.drop('level_3',axis=1, inplace=True)

## 3. Calculate growth rate 

1. Calculate the difference between length in adjacent rows in order to determine when a division occurs. If the difference is below a threshold of -0.75, then a division occurs (1 in the column 'drop')

In [20]:
data_stacked['diff_length'] = data_stacked['MajAx'].diff()

In [21]:
dropThreshold = -0.75
data_stacked['drop'] = np.where(data_stacked['diff_length']<dropThreshold, 1, 0)

2. Calculate volume as a cylinder with hemispherical caps

In [22]:
def calculate_volume(w, l):
    small_cyl = (np.pi*(w/2)**2)*(l - w)
    sphere_vol = 4/3 * np.pi * np.power((w/2), 3)
    volume = small_cyl + sphere_vol
    return volume

In [23]:
data_stacked['Volume'] = (data_stacked.apply(lambda x: 
                        calculate_volume(x['MinAx'],x['MajAx']), axis=1))

In [24]:
data_stacked

,particleID,movieID,TrackID,X,Y,Area,Intensity,Frame,Ecc,MajAx,MinAx,Ang,diff_length,drop,Volume
0,1,1,1,126.193415,98.940419,3.016181,-32.522974,2.0,0.915804,3.128856,1.256631,-25.476072,NaN,0,3.361024
1,1,1,1,126.337843,98.351280,3.027917,-32.451960,3.0,0.929216,3.269045,1.208036,-26.992272,0.140189,0,3.285348
2,1,1,1,126.375513,98.311793,3.145278,-32.423334,4.0,0.930460,3.340498,1.223935,-26.269621,0.071453,0,3.450227
3,1,1,1,126.466954,99.204797,3.297847,-31.837148,5.0,0.933886,3.475664,1.242798,-26.605013,0.135166,0,3.713736
4,1,1,1,126.531139,99.191278,3.426944,-32.593794,6.0,0.938763,3.607290,1.242940,-27.054757,0.131626,0,3.874235
5,1,1,1,126.505446,99.080401,3.509097,-32.472349,7.0,0.945311,3.759039,1.226087,-26.600524,0.151749,0,3.955685
6,1,1,1,126.472932,99.094372,3.602986,-31.747854,8.0,0.949435,3.892875,1.222225,-27.310629,0.133836,0,4.089340
7,1,1,1,126.519313,99.206089,3.779028,-32.114233,9.0,0.952625,4.072498,1.238640,-26.835052,0.179623,0,4.409767
8,1,1,1,126.574133,99.269397,3.919861,-32.096600,10.0,0.954131,4.189881,1.254405,-27.197475,0.117383,0,4.661308
9,1,1,1,126.691546,99.449154,4.013750,-31.940119,11.0,0.956913,4.284252,1.244035,-25.730113,0.094371,0,4.703471


3. Set up time dataframe

Create a dataframe with the matrix downloaded from the mat file. Flatten the numpy arrays and arrange the data in the same way as with the particle measurments.

In [108]:
timedat = pd.DataFrame(time)

In [109]:
timedat = timedat.applymap(lambda x: x.flatten())

In [110]:
timedat.reset_index(inplace=True)
timedat = timedat.rename(index=str, columns={'index': 'movieID', 0:'time'})

In [111]:
timedat['movieID'] = timedat['movieID'].apply(lambda x: x +1)

In [112]:
timedat = timedat.set_index('movieID')

In [113]:
timelist = []
for col in timedat.columns:
    timelist.append(timedat[col].apply(pd.Series).stack())
time_stacked = pd.concat(timelist, axis=1, keys=timedat.columns)

In [114]:
time_stacked.reset_index(inplace=True)
time_stacked = time_stacked.rename(index=str, columns={'level_1':'Frame'})
time_stacked['Frame'] = time_stacked['Frame'].apply(lambda x: x+1)

4. Create a new dataframe that will contain the growth rates. Merge the relevant info from the data_stacked and the time_stacked dataframes. The identifiers that link each measurement are: Frame and movieID.

In [180]:
gr_dat = (data_stacked[['particleID','movieID','TrackID','Frame','drop','Volume']]
          .merge(time_stacked, on=['movieID','Frame']))

Calculate the dV by calculating the difference between adjacent values. Do the same for the time.

In [181]:
gr_dat['dV_log'] = gr_dat.groupby(['movieID','particleID'])['Volume'].transform(pd.Series.diff)

In [182]:
gr_dat['dt'] = gr_dat.groupby(['movieID','particleID'])['time'].transform(pd.Series.diff)

In [183]:
gr_dat['GR'] = gr_dat['dV_log']/gr_dat['dt']*3600/np.log(2)

Replace all the growth rates at division events with NaN i.e. look for the drop values = 1

In [192]:
gr_dat.loc[gr_dat['drop']==1, 'GR'] = np.nan

In [193]:
gr_dat

,particleID,movieID,TrackID,Frame,drop,Volume,time,dV_log,dt,GR
0,1,1,1,2.0,0,3.361024,120.515184,NaN,NaN,NaN
1,2,1,2,2.0,1,4.069741,120.515184,NaN,NaN,NaN
2,3,1,4,2.0,1,2.529088,120.515184,NaN,NaN,NaN
3,4,1,5,2.0,0,3.383588,120.515184,NaN,NaN,NaN
4,5,1,6,2.0,1,3.973511,120.515184,NaN,NaN,NaN
5,6,1,8,2.0,0,4.817728,120.515184,NaN,NaN,NaN
6,7,1,9,2.0,0,8.455646,120.515184,NaN,NaN,NaN
7,8,1,10,2.0,1,3.556219,120.515184,NaN,NaN,NaN
8,9,1,11,2.0,1,2.948992,120.515184,NaN,NaN,NaN
9,10,1,12,2.0,1,2.899784,120.515184,NaN,NaN,NaN
